<a href="https://colab.research.google.com/github/mr20143/Script/blob/main/Custom_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Define the path to the training directory
train_dir = '/content/drive/MyDrive/Ed Folder/Kaggle_2_class - mixed/chest_xray/train'

# Get the list of class directories
class_dirs = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]

# Define the path to the output file
output_file_path = '/content/drive/MyDrive/test_counts.txt'

# Open the output file for writing
with open(output_file_path, 'w') as output_file:
    # Count the number of images for each class
    class_counts = {}
    for class_dir in class_dirs:
        class_path = os.path.join(train_dir, class_dir)
        num_images = len(os.listdir(class_path))
        class_counts[class_dir] = num_images

    # Write the counts for each class to the file
    for class_name, count in class_counts.items():
        output_file.write(f'Class: {class_name}, Number of Images: {count}\n')

# Print confirmation message
print(f"Class counts have been written to '{output_file_path}'.")

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np
import sys

# Define a function to print to both console and file
def print_and_write(text, file):
    print(text)
    file.write(text + '\n')

# Redirect standard output to the file
log_file_path = '/content/drive/MyDrive/training_logs.txt'
log_file = open(log_file_path, 'w')
sys.stdout = log_file

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_data = datasets.ImageFolder('/content/drive/MyDrive/Ed Folder/Kaggle_2_class - mixed/chest_xray/train', transform=transform)
val_data = datasets.ImageFolder('/content/drive/MyDrive/Ed Folder/Kaggle_2_class - mixed/chest_xray/val', transform=transform)

# Define the percentage of data to be used for training (50%)
train_size = int(1 * len(train_data))
indices = list(range(len(train_data)))
np.random.shuffle(indices)
train_idx = indices[:train_size]

# Create data loaders with SubsetRandomSampler
train_loader = DataLoader(train_data, batch_size=16, sampler=SubsetRandomSampler(train_idx))
val_loader = DataLoader(val_data, batch_size=16)

# Load pretrained DenseNet model
model = torchvision.models.densenet121(pretrained=True)

# Modify the model for binary classification
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Training loop
num_epochs = 40
for epoch in range(num_epochs):
    # Train the model
    model.train()
    train_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

        # Calculate accuracy
        preds = (outputs > 0.5).float()
        correct_predictions += (preds == labels.float().unsqueeze(1)).sum().item()
        total_samples += labels.size(0)

    train_loss /= len(train_loader.dataset)
    train_accuracy = correct_predictions / total_samples

    # Validate the model
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f'Validation'):
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            val_loss += loss.item() * inputs.size(0)
            preds = (outputs > 0.5).float()
            val_preds.extend(preds.cpu().numpy())
            val_targets.extend(labels.cpu().numpy())
    val_loss /= len(val_loader.dataset)
    val_accuracy = accuracy_score(val_targets, val_preds)

    # Step the scheduler
    scheduler.step()

    # Print epoch statistics
    print_and_write(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Learning Rate: {scheduler.get_last_lr()[0]}', log_file)

# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/trained_model.pth')

# Close the file
log_file.close()

# Restore standard output
sys.stdout = sys.__stdout__


In [ ]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

# Define transforms for the test dataset (make sure they match the ones used during training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_data = datasets.ImageFolder('/content/drive/MyDrive/Ed Folder/Kaggle_2_class - mixed/chest_xray/train', transform=transform)

# Create a data loader for the test dataset
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

# Load the pre-trained DenseNet model
model = torchvision.models.densenet121(pretrained=False)  # Make sure pretrained is set to False
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)

# Load the saved weights of your trained model
model.load_state_dict(torch.load('/content/drive/MyDrive/trained_model.pth'))

# Set the model to evaluation mode
model.eval()

# Define a criterion for evaluation (e.g., Binary Cross Entropy Loss)
criterion = nn.BCELoss()

# Define lists to store predictions and ground truth labels
predictions = []
targets = []
losses = []

# Iterate over the test dataset and make predictions
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        preds = (outputs > 0.5).float()
        predictions.extend(preds.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        losses.append(loss.item())  # Append the loss value

# Calculate accuracy
accuracy = accuracy_score(targets, predictions)

# Calculate the average loss
average_loss = sum(losses) / len(losses)

# Print test accuracy and loss to a log file
log_file_path = '/content/drive/MyDrive/test_logs.txt'
with open(log_file_path, 'w') as log_file:
    log_file.write(f'Test Accuracy: {accuracy:.4f}\n')
    log_file.write(f'Average Loss: {average_loss:.4f}\n')
